Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#creating a dataframe df which read and will contain the file
df  = pd.read_csv(r"C:\Users\HP\Desktop\ML projects\T_20 cricket Prediction\ball_by_ball_it20.csv")

In [ ]:
df.head()

In [ ]:
#Checking for nnull Values
df.isnull().sum()

In [ ]:
#Printing all the column names so that we can select the important ones on which we will proceed our work
df.columns

In [ ]:
df.head(1)

In [ ]:
#Seperating the dataset into three so that each column is easily visible
data = {
    f'col{i}': range(10) for i in range(1, 36)
}

# Splitting the DataFrame into three smaller DataFrames
df1 = df.iloc[:, :12]
df2 = df.iloc[:, 12:24]
df3 = df.iloc[:, 24:35]

In [ ]:
df1.head(8)

In [ ]:
df2.head()

In [ ]:
#df3['Chased Successfully'].value_counts()
df3.head(1)

Now I created two files Matches and Deliveries, as per need

In [ ]:
matches = df[['Match ID','Venue','Winner','Bat First','Bat Second','Target Score']]

In [ ]:
#matches.head()

In [ ]:
deliveries = df[['Match ID','Venue','Innings','Bat First','Bat Second','Batter','Winner','Non Striker','Bowler','Ball','Batter Runs','Extra Runs','Runs From Ball','Ball Rebowled','Extra Type','Wicket','Method','Player Out','Innings Runs','Innings Wickets','Target Score','Runs to Get'
	,'Balls Remaining']]

In [ ]:
deliveries.head(785)

Data cleaning and PREPROCESSING

In [ ]:
matches['Bat First'].unique()

In [ ]:
#Removing non playing Nation and selected which are playing presently and specially taking part in T_20 WC 2024
Teams = ['West Indies', 'Scotland',
       'Sri Lanka', 'England',
       'United Arab Emirates', 'Denmark', 'Bangladesh',
       'India', 'Papua New Guinea', 'Afghanistan', 
       'Netherlands', 'Ireland', 'New Zealand','Pakistan',
      'Nepal', 'South Africa', 'Australia',
       'Oman','Zimbabwe', 'Hong Kong', 'Kenya', 'Canada','Namibia',
       'United States of America', 
      ]

In [ ]:
len(Teams)

Updating files as per selected teams 

In [ ]:
matches = matches[matches['Bat First'].isin(Teams)]
matches = matches[matches['Bat Second'].isin(Teams)]

In [ ]:
deliveries = deliveries[deliveries['Bat Second'].isin(Teams)]
deliveries = deliveries[deliveries['Bat First'].isin(Teams)]

In [ ]:
deliveries = deliveries[deliveries['Innings'] == 2]

In [ ]:
deliveries.shape

In [ ]:
deliveries = deliveries.rename(columns = {'Innings Runs': 'Current Score','Innings Wickets':'Wickets','Runs to Get':'Required Runs','Runs From Ball':'Run scored'})

In [ ]:
deliveries.head()

In [ ]:
#Creating new column wicket left,and droping unnecessary columns
deliveries = deliveries.drop(columns = {"Method","Player Out"})
deliveries["Wickets Left"] = 10 - deliveries['Wickets']

In [ ]:
deliveries.head()

In [ ]:
#Now droping wicket as we only require wickets left
deliveries = deliveries.drop(columns = {"Wicket"})

In [ ]:
deliveries.head()

Creating new column for Required run-rate and Current run-rate

In [ ]:
deliveries['RR'] = (deliveries['Required Runs']/deliveries['Balls Remaining'])*6
deliveries['CR'] = (deliveries['Current Score']/(120 - deliveries['Balls Remaining'])*6)

In [ ]:
deliveries.head(6)

In [ ]:
# for team in deliveries['Bat Second']:
#     if team == deliveries['Winner']:
#         deliveries['Result'] = 1
#     else :
#         deliveries['Result'] =0

In [ ]:
#Creating another column Result which contain 0 and 1 to indicate match result
deliveries['Result'] = deliveries.apply(lambda row: 1 if row['Bat Second'] == row['Winner'] else 0, axis=1)

In [ ]:
deliveries.head(1)

In [ ]:
#droping NA values
deliveries.dropna(inplace=True)

In [ ]:
#Removing rows which have 0 ball remaining in it as it create problem in run-rate
deliveries = deliveries[deliveries['Balls Remaining'] != 0]

In [ ]:
deliveries.shape

In [ ]:
#Renaming some columns for convinience
deliveries = deliveries.rename(columns = {'Bat First':'Bowling Side','Bat Second': 'Batting Side'})

In [ ]:
#deliveries.head()

Finally selecting columns for training and testing

In [ ]:
Final_doc = deliveries[['Bowling Side','Batting Side','Venue','Target Score','Required Runs','Balls Remaining','CR','RR','Wickets Left','Result']]

In [ ]:
Final_doc.dropna(inplace=True)
Final_doc = Final_doc[Final_doc['Balls Remaining'] != 0]
Final_doc = Final_doc[Final_doc['Wickets Left'] != 0]
Final_doc.head()

In [ ]:
#Sving file
#Final_doc.to_csv('T_20i.csv')

In [ ]:
deliveries.to_csv('Deliveries_1.csv')

In [ ]:
Final_doc.replace([np.inf, -np.inf, np.finfo(np.float64).max], np.nan, inplace=True)
Final_doc.dropna(inplace=True)

In [ ]:
X = Final_doc.drop(columns = "Result")
Y = Final_doc.Result

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,Y,test_size = 0.2,random_state = 22)
#X.head(1)
#Y.head(1)
# X_train.head()
# y_train.head()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse_output=False,drop='first'),['Batting Side','Bowling Side','Venue','Wickets Left'])
]
,remainder='passthrough')


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [ ]:
# clf = Pipeline(steps=[
#     ('step1',trf),
#     ('step2',LogisticRegression(solver='liblinear'))
# ])
from sklearn.tree import DecisionTreeClassifier

clf = Pipeline(steps=[
    ('step1', trf),
    ('step2', DecisionTreeClassifier(random_state=42))
])

# from sklearn.ensemble import RandomForestClassifier

# clf = Pipeline(steps=[
#     ('step1', trf),
#     ('step2', RandomForestClassifier(n_estimators=100, random_state=42))
# ])

from sklearn.svm import SVC

clf = Pipeline(steps=[
    ('step1', trf),
    ('step2', SVC(probability=True, random_state=42))
])


In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,y_pred))

Now just checking predictions for some matches and it works 

In [ ]:
def match_summary(row):
    print("Batting Team-" + row['Btting Side'] + " | Bowling Team-" + row['Bowling Side'] + " | Target- " + str(row['Target Score']))


In [ ]:
def match_progression(x_df,match_id,pipe):
    match = x_df[x_df['Match ID'] == match_id]
    match = match[(match['Ball'] == 6)]
    temp_df = match[['Batting Side','Bowling Side','Venue','Required Runs','Balls Remaining','Wickets Left','Target Score','CR','RR']].dropna()
    temp_df = temp_df[temp_df['Balls Remaining'] != 0]
    result = pipe.predict_proba(temp_df)
    temp_df['lose'] = np.round(result.T[0]*100,1)
    temp_df['win'] = np.round(result.T[1]*100,1)
    temp_df['end_of_over'] = range(1,temp_df.shape[0]+1)
    
    target = temp_df['Target Score'].values[0]
    runs = list(temp_df['Required Runs'].values)
    new_runs = runs[:]
    runs.insert(0,target)
    temp_df['runs_after_over'] = np.array(runs)[:-1] - np.array(new_runs)
    wickets = list(temp_df['Wickets Left'].values)
    new_wickets = wickets[:]
    new_wickets.insert(0,10)
    wickets.append(0)
    w = np.array(wickets)
    nw = np.array(new_wickets)
    temp_df['wickets_in_over'] = (nw - w)[0:temp_df.shape[0]]
    
    print("Target-",target)
    temp_df = temp_df[['end_of_over','runs_after_over','wickets_in_over','lose','win']]
    return temp_df,target

In [ ]:
temp_df,target = match_progression(deliveries,1298176,clf)
temp_df

In [ ]:
deliveries['Venue'].unique()

In [2]:
import joblib
import pandas as pd

# Load the model
clf = joblib.load("clf.joblib")

In [ ]:
deliveries['Batting Side'].unique()

I trained my model on linear regression, it can be done on some other models as well 
like:-
1.Logistic Regression
2.Decision Trees
3.Random Forests
4.Gradient Boosting Machines (GBM)
etc. 


THANKS FOR VISITING